Implementa algoritmo Held-Karp per risolvere il problema del comesso viaggiattore in modo ottimale

In [164]:
import sys
sys.path.append('..')

from WorldParser import WorldParser
from GeneticAlgorithm import GeneticAlgorithm
import networkx as nx


des_file = """
MAZE: "mylevel", ' '
FLAGS:premapped
GEOMETRY:center,center

MAP
|||||||||||||
|...........|
|..|........|
|...........|
|.......|...|
|.....|.....|
|...|....|..|
|||||||||||||
ENDMAP

BRANCH:(1,1,1,1),(0,0,0,0)
GOLD:1,(1,2)
GOLD:1,(2,1)
GOLD:1,(7,4)
GOLD:1,(9,4)
GOLD:1,(11,5)
GOLD:1,(6,1)

"""

starting_position = (1,1)

In [165]:
def calculate_distances(G, points):
    """
    Restituisce un dizionare dove per ogni punto calcola la distanza minima con tutti gli altri punti.
    """
    
    distances = {}
    for point in points:
        lengths = nx.single_source_dijkstra_path_length(G, point)
        distances[point] = {other: lengths[other] for other in points if other in lengths}
    return distances

In [166]:
parser = WorldParser(des_file)
gold_positions = parser.extract_gold_positions()
world_graph = parser.extract_world_graph()

In [167]:
start = (1, 1)
world_graph = parser.extract_world_graph()
distances = calculate_distances(world_graph, gold_positions + [start])

dist = [[distances[i][j] for j in distances[i]] for i in distances] #matrice delle distanze
n = len(dist) -1

In [168]:
# This code is contributed by Serjeel Ranjan (https://www.geeksforgeeks.org/traveling-salesman-problem-tsp-in-python/)

# memoization for top down recursion
memo = [[-1]*(1 << (n+1)) for _ in range(n+1)]


def fun(i, mask):
    # base case
    # if only ith bit and 1st bit is set in our mask,
    # it implies we have visited all other nodes already
    if mask == ((1 << i) | 3):
        return dist[1][i]

    # memoization
    if memo[i][mask] != -1:
        return memo[i][mask]

    res = 10**9  # result of this sub-problem

    # we have to travel all nodes j in mask and end the path at ith node
    # so for every node j in mask, recursively calculate cost of
    # travelling all nodes in mask
    # except i and then travel back from node j to node i taking
    # the shortest path take the minimum of all possible j nodes
    for j in range(1, n+1):
        if (mask & (1 << j)) != 0 and j != i and j != 1:
            res = min(res, fun(j, mask & (~(1 << i))) + dist[j][i])
    memo[i][mask] = res  # storing the minimum value
    return res


# Driver program to test above logic
ans = 10**9
for i in range(1, n+1):
    # try to go from node 1 visiting all nodes in between to i
    # then return from i taking the shortest route to 1
    ans = min(ans, fun(i, (1 << (n+1))-1) + dist[i][1]) + 1

print("The cost of most efficient tour = " + str(ans))

The cost of most efficient tour = 21
